In [4]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

# For mac OS: Enter 'brew install libmagic' in terminal if magic module is unavailable

In [10]:
data_all = pd.read_csv(r"D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_MEAN.csv")
over_adult = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_30A.csv')
under_adult = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_18A.csv')
over_child = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_PLUS1C.csv')
under_child = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_MINUS2C.csv')

C:\Users\eball\AppData\Local\Temp\ipykernel_6936\2054899572.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  over_child = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_PLUS1C.csv')
C:\Users\eball\AppData\Local\Temp\ipykernel_6936\2054899572.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  under_child = pd.read_csv(r'D:\lina_lau\C339_datafundamentals\data_analysis_project\bmi_dashboard\dataset\NCD_BMI_MINUS2C.csv')


In [ ]:
under_adult.info()

In [16]:
data_all.columns

Index(['Id', 'IndicatorCode', 'SpatialDimension', 'SpatialDimensionValueCode',
       'TimeDimension', 'TimeDim', 'DisaggregatingDimension1',
       'DisaggregatingDimension1ValueCode', 'DisaggregatingDimension2',
       'DisaggregatingDimension2ValueCode', 'DisaggregatingDimension3',
       'DisaggregatingDimension3ValueCode', 'DataSourceDimension',
       'DataSourceDimensionValueCode', 'Value', 'NumericValue', 'Low', 'High',
       'Comments', 'Date', 'TimeDimensionValue', 'TimeDimensionBegin',
       'TimeDimensionEnd'],
      dtype='object')

In [18]:
#data_all_incomegroup = data_all[data_all['SpatialDimension'] == 'WORLDBANKINCOMEGROUP']

In [11]:
# Calculating the proportion of missing values in each column of the dataset
data_all.isnull().sum() * 100 / len(data_all)

Id                                     0.000000
IndicatorCode                          0.000000
SpatialDimension                       0.000000
SpatialDimensionValueCode              0.000000
TimeDimension                          0.000000
TimeDim                                0.000000
DisaggregatingDimension1               0.000000
DisaggregatingDimension1ValueCode      0.000000
DisaggregatingDimension2               0.000000
DisaggregatingDimension2ValueCode      0.000000
DisaggregatingDimension3             100.000000
DisaggregatingDimension3ValueCode    100.000000
DataSourceDimension                  100.000000
DataSourceDimensionValueCode         100.000000
Value                                  0.000000
NumericValue                           1.929851
Low                                    1.929851
High                                   1.929851
Comments                              98.070149
Date                                   0.000000
TimeDimensionValue                     0

In [12]:
# Make dataframe pipeline interactive
idf_all = data_all.interactive()
idf_over_adult = over_adult.interactive()
idf_under_adult = under_adult.interactive()

In [15]:
# Defining Panel widgets: the year slider
year_slider = pn.widgets.IntSlider(name = 'Year slider', start = 1975, end = 2016, value = 1975)
year_slider

ModuleNotFoundError: No module named 'jupyter_bokeh'

IntSlider(end=2016, name='Year slider', start=1975, value=1975)

In [72]:
# Defining panel widgets: the sex dropdown menu
sex_menu = pn.widgets.Select(options = ['BTSX', 'FMLE', 'MLE'], name = 'DisaggregatingDimension1ValueCode')
sex_menu

Select(name='DisaggregatingDimension1V..., options=['BTSX', 'FMLE', 'MLE'], value='BTSX')

In [73]:
# Defining Panel widgets: radio buttons for mean BMI in the pooled population
yaxis_prevalence = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['NumericValue', 'Low', 'High',],
    button_type = 'success'
)

In [75]:
# Connecting the widgets to the data_all pipeline
income_group = ['WB_HI', 'WB_LI', 'WB_LMI', 'WB_UMI']

bmi_all_pipeline = (
    idf_all[
        (idf.TimeDim <= year_slider) &
        (idf.DisaggregatingDimension1ValueCode == sex_menu) & 
        (idf.SpatialDimensionValueCode.isin(income_group))
    ]
    .groupby(['SpatialDimensionValueCode', 'TimeDim'])[yaxis_prevalence].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'TimeDim')
    .reset_index(drop = True)
)

In [76]:
bmi_all_pipeline

In [77]:
# Creating a chart using the data pipeline
bmi_all_plot = bmi_all_pipeline.hvplot(x = 'TimeDim', 
                                       by = 'SpatialDimensionValueCode', 
                                       y = yaxis_prevalence, 
                                       line_width = 2, 
                                       title = 'Mean BMI of global population')
bmi_all_plot

In [78]:
# Connecting the widgets to the over_adult (overweight adults) pipeline
income_group = ['WB_HI', 'WB_LI', 'WB_LMI', 'WB_UMI']

bmi_over_adult_pipeline = (
    idf_over_adult[
        (idf.TimeDim <= year_slider) & 
        (idf.DisaggregatingDimension1ValueCode == sex_menu) & 
        (idf.SpatialDimensionValueCode.isin(income_group))
    ]
    .groupby(['SpatialDimensionValueCode', 'TimeDim'])[yaxis_prevalence].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'TimeDim')
    .reset_index(drop = True)
)

In [52]:
bmi_over_adult_pipeline

In [83]:
# Creating a chart for overweight adults
bmi_over_adult_plot = bmi_over_adult_pipeline.hvplot(x = 'TimeDim', 
                                       by = 'SpatialDimensionValueCode', 
                                       y = yaxis_prevalence, 
                                       line_width = 2, 
                                       title = 'Age-standardised prevalence of overweight adults (BMI >30 kg/m2) by income groups', 
                                       width = 800)
bmi_over_adult_plot

In [84]:
# Connecting the widgets to the under_adult (underweight adults) pipeline
income_group = ['WB_HI', 'WB_LI', 'WB_LMI', 'WB_UMI']

bmi_under_adult_pipeline = (
    idf_under_adult[
        (idf.TimeDim <= year_slider) & 
        (idf.DisaggregatingDimension1ValueCode == sex_menu) & 
        (idf.SpatialDimensionValueCode.isin(income_group))
    ]
    .groupby(['SpatialDimensionValueCode', 'TimeDim'])[yaxis_prevalence].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'TimeDim')
    .reset_index(drop = True)
)

In [85]:
bmi_under_adult_pipeline

In [100]:
# Creating a chart for underweight adults
bmi_under_adult_plot = bmi_under_adult_pipeline.hvplot(x = 'TimeDim', 
                                       by = 'SpatialDimensionValueCode', 
                                       y = yaxis_prevalence, 
                                       xlabel = "Years",
                                       ylabel = "Prevalence (%)",
                                       line_width = 2, 
                                       title = 'Age-standardised prevalence of underweight adults (BMI <18 kg/m2) by income groups', 
                                       width = 800, height = 400)
bmi_under_adult_plot

In [101]:
# Creating the dashboard
template = pn.template.FastListTemplate(
    title = 'Global Obesity Prevalence',
    sidebar = [pn.pane.Markdown("Global prevalence of overweight and underweight people"),
              pn.pane.Markdown("#### Increased income and globalisation has led to an explosion of readily-available foods."),
              pn.pane.JPG('fast-food-junk-food-concept.jpg', sizing_mode = 'scale_both'),
              pn.pane.Markdown("##Settings"),
              year_slider,
              sex_menu,
              pn.pane.Markdown("#### Data used for this analysis is from ")],
    main = [pn.Row(pn.Column(yaxis_prevalence, bmi_all_plot.panel(width = 800, height = 300), margin = (0, 25))),
            pn.Row(pn.Column(yaxis_prevalence, bmi_over_adult_plot.panel(width = 800, height = 300), margin = (0, 25))),
            pn.Row(pn.Column(yaxis_prevalence, bmi_under_adult_plot.panel(width = 800, height = 300), margin = (0, 25)))],
    accent_base_color = "#88d8b0",
    header_base_color = "#88d8b0",
)
template.show()
#template.servable()

Launching server at http://localhost:53117
